In [ ]:
!pip install pandas
!pip install folium
!pip install gpxpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import gpxpy
import folium
import pandas as pd

In [ ]:
import pandas as pd

file_path = "/content/drive/My Drive/BMB_GIS/marci/own_projects/hun_tour_2025/csv_locations_proba/input/loc_jo_1.csv"  # Adjust the path
df = pd.read_csv(file_path, encoding="windows-1250", sep=";")


print(df.head())
print(df.dtypes)


df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

print(df.dtypes)  # Check if they are now float64

print(df[df["latitude"].isna() | df["longitude"].isna()])  # Show rows with NaN values

   julius        telepules   latitude  longitude
0       1  Sátoraljaújhely  48.400834  21.657103
1       2             Gönc  48.473848  21.275848
2       3  Bodrogkeresztúr  48.159389  21.361834
3       4     Tiszaújváros  47.931012  21.042745
4       6        Nyírlugos  47.657326  22.035925
julius         int64
telepules     object
latitude     float64
longitude    float64
dtype: object
julius         int64
telepules     object
latitude     float64
longitude    float64
dtype: object
Empty DataFrame
Columns: [julius, telepules, latitude, longitude]
Index: []


# első próba

In [ ]:
import folium

# Create a Folium map centered at the average location
map_center = [df["latitude"].mean(), df["longitude"].mean()]
mymap = folium.Map(location=map_center, zoom_start=6)

# Add multiple points from CSV
for _, row in df.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["telepules"],  # Label shown when clicked
        tooltip=row["julius"],  # Label shown on hover
    ).add_to(mymap)

# Save the map as an HTML file
mymap.save("hun_tour_map.html")

print("Map saved as map.html! You can open it in a browser.")

Map saved as map.html! You can open it in a browser.


# javított verzió

In [ ]:
import folium

# Create a Folium map centered at the average location
map_center = [df["latitude"].mean(), df["longitude"].mean()]
mymap = folium.Map(location=map_center, zoom_start=6)


#-----------------------------------------Adding hun_admin2----------------------



# URL of your GeoJSON file hosted on GitHub
geojson_megye = "https://raw.githubusercontent.com/wuerdo/geoHungary/refs/heads/master/counties.geojson"


# Define the style function to customize the GeoJSON layer
def style_function(feature):
    return {
        "fillColor": "transparent",  # No fill color
        "color": "black",  # Contour line color
        "weight": 2,  # Contour line thickness
        "opacity": 0.5  # Contour line transparency
    }


# Add the GeoJSON layer
#folium.GeoJson(geojson_url, name="geojson").add_to(mymap)

# Add the GeoJSON layer with the style function
folium.GeoJson(
    geojson_megye,
    name="Megyék",
    style_function=style_function
).add_to(mymap)

#--------------------------------------------------------------


#geojson_utak = "https://raw.githubusercontent.com/baloghmb98/data/refs/heads/main/hun_tour_2025/geojson/utak_proba.geojson"
#folium.GeoJson(geojson_utak, name="Utak").add_to(mymap)

#geojson_data_validation = "https://raw.githubusercontent.com/baloghmb98/data/refs/heads/main/hun_tour_2025/geojson/GSV_IP_data_validation_geojson.geojson"
#folium.GeoJson(geojson_data_validation, name="validation_points").add_to(mymap)


# Add numbered points from the 'julius' column
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        popup=row["telepules"],  # Label shown when clicked
        tooltip=row["telepules"],  # Label shown on hover
        radius=10,  # Adjust size
        color="black",
        fill=True,
        fill_color="yellow",
        fill_opacity=0.7,
    ).add_to(mymap)

    # Add a label with the number from 'julius'
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 10px; color: black; font-weight: bold;">{row["julius"]}</div>'
        ),
    ).add_to(mymap)


#----------------------------------------------------Adding .gpx tracks

# Loop through folders like 'bus', 'train', 'walking' and draw their tracks
base_path = '/content/drive/My Drive/BMB_GIS/marci/own_projects/hun_tour_2025/csv_locations_proba/input/gpx_proba_files'
transport_colors = {
    'bus': 'red',
    'train': 'purple',
    'walking': 'green',
    'car': 'brown',
    'fahrrad': 'orange'
}

for transport_type, color in transport_colors.items():
    folder_path = os.path.join(base_path, transport_type)
    for filename in os.listdir(folder_path):
        if filename.endswith('.gpx'):
            with open(os.path.join(folder_path, filename), 'r') as gpx_file:
                gpx = gpxpy.parse(gpx_file)
                for track in gpx.tracks:
                    for segment in track.segments:
                        coords = [(pt.latitude, pt.longitude) for pt in segment.points]
                        folium.PolyLine(
                            coords,
                            color=color,
                            weight=3,
                            opacity=0.8,
                            tooltip=f'{transport_type}: {filename}'
                        ).add_to(mymap)

#---------------------------------------------------------------------------------------------



#-------------------------------Add layer control to toggle layers
folium.LayerControl().add_to(mymap)
#--------------------------------------


# Save the map as an HTML file
mymap.save("HT_map_4_proba.html")
print("Map saved as map.html! You can open it in a browser.")

Map saved as map.html! You can open it in a browser.


In [ ]:
save_path = "/content/drive/My Drive/BMB_GIS/marci/own_projects/hun_tour_2025/csv_locations_proba/output/HT_map_4.html"  # Adjust the folder path
mymap.save(save_path)
print(f"Map saved to Google Drive: {save_path}")

Map saved to Google Drive: /content/drive/My Drive/BMB_GIS/marci/own_projects/hun_tour_2025/csv_locations_proba/output/HT_map_4.html
